In [1]:
import sys
sys.path[0]+='/..'
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset
from model.tripartite_4 import GNNPolicy
from utilities import *
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
from uc_class import UC

import gurobipy as gp
import json
from torch_geometric.data import HeteroData
import collections


In [2]:
#set folder
uc_model = '3bin_1bin_startupcost'
# instance = '60_c11_based_8_std'
instance = '80_c11_based_8_std'
# instance = '5_std'
model = 'tripartite_lpsolve'



model_save_path = f'../model_save/{uc_model}_{instance}_{model}/'
log_save_path =   f"../model_save/{uc_model}_{instance}_{model}_log/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-4
NB_EPOCHS =1000
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-5

In [4]:
train_data = GraphDataset(f'../datasets/{instance}/{uc_model}/tripartite_lpsolve/train/')
valid_data = GraphDataset(f'../datasets/{instance}/{uc_model}/tripartite_lpsolve/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict)
    print("载入模型成功")

载入模型成功


In [6]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    acc = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            if optimizer:
                weighted_sol = batch.weighted_sol
            else:
                weighted_sol = batch.opt_sol
            presol = predict(batch).squeeze(1)
            
            loss = loss_fn(presol, weighted_sol)
            preds = (presol > 0.4).float()  # Assuming threshold of 0.5
            acc += torch.sum(preds == batch.opt_sol).item()/len(batch.opt_sol)

            
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed
    acc /= n_samples_processed
    return mean_loss,acc


In [7]:
def valid(predict,instance_set):
    # instance_set = ['80_c11_based_8_std','1080_c30_based_8_std']

    opt_loss=[]
    for ins in instance_set:

        uc_path = f'../UC_AF/{ins}.mod'
        uc = UC(uc_path)
        json_path = f'../datasets/{ins}/json/valid/'
        inf_cnt=0
        files = os.listdir(json_path)
        files = [file for file in files if file.endswith('.json')][::3]
        for i,file in enumerate(files):
            with open(os.path.join(json_path, file),'r') as f:
                data = json.load(f)
            #参数
            pmax = uc.ThPimax
            Dt = data['Dt']
            Spin = data['Spin']
            u0 = data['u0']
            p0 = data['p0']
            on_off = data['on_off']
            #
            m3 = uc.get_3bin_model_1bin_startupcost(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
            m1 = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)

            graph=get_Tripartite_graph_lp(m3).to(DEVICE)
            sol = predict(graph).detach().cpu().numpy().reshape(-1,24)
            sol = restore(sol,uc,data).reshape(-1).astype(int)
            obj=solve(m1,sol)
            opt_obj = data['obj']
            if obj == -1:
                inf_cnt+=1
            else:
                opt_loss.append((obj-opt_obj)/obj)
        if len(files)==0:
            print('no file')
    opt_loss=np.array(opt_loss)
    return opt_loss.mean()

In [8]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS/10, eta_min=ETA_MIN)
best_val_loss = 1
best_optloss=1
cur_epoch = 0
instance_set = ['80_c11_based_8_std']

In [9]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss,train_acc = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f} Train acc:{train_acc:0.6f}",end='\t')
    valid_loss,valid_acc = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f} Valid acc:{valid_acc:0.6f}")
    optloss=valid(PredictModel,instance_set)
    print("optloss",optloss)
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    if cur_epoch%20==0:
        torch.save(PredictModel.state_dict(),model_save_path+f'model_{cur_epoch}.pth')
    if optloss<best_optloss:
        best_optloss=optloss
        print("best_optloss",best_optloss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best_acc.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}  Valid acc:{valid_acc} optloss:{optloss} TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.122033 Train acc:0.961053	Epoch 0 Valid loss: 0.114490 Valid acc:0.962441
optloss 0.007656125699043238
best_loss 0.11449030125993216
best_optloss 0.007656125699043238
Epoch 1 Train loss: 0.121432 Train acc:0.961059	Epoch 1 Valid loss: 0.113751 Valid acc:0.961938
optloss 0.007449172217394272
best_loss 0.11375081390243397
best_optloss 0.007449172217394272
Epoch 2 Train loss: 0.120778 Train acc:0.961129	Epoch 2 Valid loss: 0.111903 Valid acc:0.962128
optloss 0.007554598375516859
best_loss 0.11190251487138149
Epoch 3 Train loss: 0.120196 Train acc:0.961134	Epoch 3 Valid loss: 0.112191 Valid acc:0.962697
optloss 0.006794641623551817
best_optloss 0.006794641623551817
Epoch 4 Train loss: 0.118449 Train acc:0.961125	Epoch 4 Valid loss: 0.109657 Valid acc:0.961143
optloss 0.00698133571101527
best_loss 0.10965676442656898
Epoch 5 Train loss: 0.115857 Train acc:0.961268	Epoch 5 Valid loss: 0.108536 Valid acc:0.961841
optloss 0.007058663052061669
best_loss 0.10853636813498932

KeyboardInterrupt: 